In [6]:
import math
from collections import defaultdict
from datetime import datetime
from sqlalchemy import sql
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from constants import party_colors
from database_utils import Session; session = Session();
from models.models import Appointment, Education, Judge, Court, YearParty, Congress, UnsuccessfulNomination

In [25]:
president_party_sq = (
    session.query(
        Appointment.party_of_appointing_president, Appointment.appointing_president
    )
    .group_by(Appointment.party_of_appointing_president, Appointment.appointing_president)
).subquery('president_party_sq')



unsuccesful_nominations_query = (
    session.query(
        UnsuccessfulNomination.president,
        president_party_sq.c.party_of_appointing_president,
        UnsuccessfulNomination.congress_start_year,
        sql.func.count(1).label('count')
    )
    .join(president_party_sq, president_party_sq.c.appointing_president == UnsuccessfulNomination.president)
    .group_by(
        UnsuccessfulNomination.president,
        president_party_sq.c.party_of_appointing_president,
        UnsuccessfulNomination.congress_start_year,
    )
    .order_by(
        UnsuccessfulNomination.congress_start_year,
        # Because some president's appoint in their lame duck period, the congress is the _next_ congress
        # Use minimum nomination date to sort it so all of president's rows are blocked together
        sql.func.min(UnsuccessfulNomination.nomination_date),
        president_party_sq.c.party_of_appointing_president,
        UnsuccessfulNomination.president,
    )
)

In [26]:
fig = go.Figure()


xs, ys, names, colors = [], [], [], []
for president, party, year, count in unsuccesful_nominations_query.all():
    xs.append(year)
    ys.append(count)
    names.append(president)
    colors.append(party_colors.get(party))
    
    
fig.add_trace(
    go.Bar(
        y=ys,
        x=xs,
#         name=names,
        # hoverinfo='name',
#         marker_color=colors,
    )
)

fig.update_layout(
    xaxis={'showticklabels': True},
    yaxis={'title': 'Number of Unsuccesful Judges'},
)

In [20]:
ys[-2]

14535

In [22]:
print(unsuccesful_nominations_query)

SELECT unsuccessful_nomination.president AS unsuccessful_nomination_president, appointment.party_of_appointing_president AS appointment_party_of_appointing_president, unsuccessful_nomination.congress_start_year AS unsuccessful_nomination_congress_start_year, count(%(count_1)s) AS count 
FROM unsuccessful_nomination JOIN appointment ON appointment.appointing_president = unsuccessful_nomination.president GROUP BY unsuccessful_nomination.president, appointment.party_of_appointing_president, unsuccessful_nomination.congress_start_year ORDER BY unsuccessful_nomination.congress_start_year, min(unsuccessful_nomination.nomination_date), appointment.party_of_appointing_president, unsuccessful_nomination.president
